In [1]:
import praw
import pandas as pd
from datetime import datetime

In [2]:
reddit = praw.Reddit(
    client_id="Pr-co5-36bkZzmAr-amBZg",
    client_secret="LhMeEf7Hy9LJS20uJWCwQQWYGWunlQ",
    user_agent="Alternative-Bag-3334"
)

In [3]:
def convert_utc_to_datetime(utc_timestamp):
    """Convert UTC timestamp to readable date and time."""
    return datetime.fromtimestamp(utc_timestamp).strftime('%Y-%m-%d %H:%M:%S')

In [4]:
def fetch_comments(submission, max_comments=10):
    # Retrieve top comments for the post
    comments_data = []
    submission.comments.replace_more(limit=0)  # Load all comments
    comments = submission.comments.list()[:max_comments]
    for comment in comments:
        if comment and hasattr(comment, 'body'):
            comments_data.append({
                'comment_body': comment.body,
                'comment_author': str(comment.author) if comment.author else "N/A",
                'comment_score': comment.score,
                'comment_created_utc': comment.created_utc,
                'comment_created_dt': convert_utc_to_datetime(comment.created_utc)
            })
    return comments_data

In [5]:
def search_reddit_posts(keyword, max_posts=5, max_comments=10):
    posts_data = []

    # Search for posts containing the keyword
    for submission in reddit.subreddit("all").search(keyword, limit=max_posts):
        if submission and hasattr(submission, 'title'):
            post = {
                'keyword': keyword,
                'title': submission.title,
                'selftext': submission.selftext if submission.selftext else "N/A",
                'url': submission.url,
                'score': submission.score,
                'num_comments': submission.num_comments,
                'created_utc': submission.created_utc,
                'created_dt': convert_utc_to_datetime(submission.created_utc),
                'author': str(submission.author) if submission.author else "N/A",
                'subreddit': str(submission.subreddit)
            }
            post['comments'] = fetch_comments(submission, max_comments)
            posts_data.append(post)

    return posts_data

In [6]:
def fetch_posts_for_keywords(keywords, max_posts_per_keyword=5, max_comments_per_post=10, output_file="reddit_posts.csv"):
    all_posts = []

    # Iterate over each keyword and fetch posts
    for keyword in keywords:
        print(f"Fetching posts for keyword: {keyword}")
        posts_data = search_reddit_posts(keyword, max_posts=max_posts_per_keyword, max_comments=max_comments_per_post)
        
        if posts_data:
            all_posts.extend(posts_data)

    # Convert all posts to a flat DataFrame format
    flat_data = []
    for post in all_posts:
        for comment in post['comments']:
            flat_data.append({
                'keyword': post['keyword'],
                'post_title': post['title'],
                'post_text': post['selftext'],
                'post_url': post['url'],
                'post_score': post['score'],
                'post_num_comments': post['num_comments'],
                'post_created_utc': post['created_utc'],
                'post_created_dt': post['created_dt'],
                'post_author': post['author'],
                'post_subreddit': post['subreddit'],
                'comment_body': comment.get('comment_body', "N/A"),
                'comment_author': comment.get('comment_author', "N/A"),
                'comment_score': comment.get('comment_score', 0),
                'comment_created_utc': comment.get('comment_created_utc', "N/A"),
                'comment_created_dt': comment.get('comment_created_dt', "N/A")
            })

    # Create DataFrame from flattened data
    posts_df = pd.DataFrame(flat_data)

    # Save DataFrame to CSV
    posts_df.to_csv(output_file, index=False)
    print(f"Data saved to {output_file}")

In [46]:
# Define keywords and execute the function
keywords = ['AI regulations in Europe']
fetch_posts_for_keywords(keywords, max_posts_per_keyword=50, max_comments_per_post=50, output_file="reddit_posts.csv")

Fetching posts for keyword: AI regulations in Europe
Data saved to reddit_posts.csv


In [47]:
df = pd.read_csv('reddit_posts.csv')

In [48]:
df

,keyword,post_title,post_text,post_url,post_score,post_num_comments,post_created_utc,post_created_dt,post_author,post_subreddit,comment_body,comment_author,comment_score,comment_created_utc,comment_created_dt
0,AI regulations in Europe,What is so limiting in EU act on AI regulations?,NaN,https://i.redd.it/3pzbsnvj0w3b1.jpg,1433,492,1.685835e+09,2023-06-03 19:27:31,No-Transition3372,ChatGPT,"Hey /u/No-Transition3372, please respond to th...",AutoModerator,1,1.685835e+09,2023-06-03 19:27:31
1,AI regulations in Europe,What is so limiting in EU act on AI regulations?,NaN,https://i.redd.it/3pzbsnvj0w3b1.jpg,1433,492,1.685835e+09,2023-06-03 19:27:31,No-Transition3372,ChatGPT,"""Wait, the regulations were supposed to hurt o...",Cressbeckler,1134,1.685848e+09,2023-06-03 23:01:42
2,AI regulations in Europe,What is so limiting in EU act on AI regulations?,NaN,https://i.redd.it/3pzbsnvj0w3b1.jpg,1433,492,1.685835e+09,2023-06-03 19:27:31,No-Transition3372,ChatGPT,Open AI wants the regulations to basically say...,AdAlternative7148,784,1.685839e+09,2023-06-03 20:38:28
3,AI regulations in Europe,What is so limiting in EU act on AI regulations?,NaN,https://i.redd.it/3pzbsnvj0w3b1.jpg,1433,492,1.685835e+09,2023-06-03 19:27:31,No-Transition3372,ChatGPT,Yeah pull it down if you don’t wanna follow th...,momolamomo,155,1.685840e+09,2023-06-03 20:47:35
4,AI regulations in Europe,What is so limiting in EU act on AI regulations?,NaN,https://i.redd.it/3pzbsnvj0w3b1.jpg,1433,492,1.685835e+09,2023-06-03 19:27:31,No-Transition3372,ChatGPT,[deleted],NaN,426,1.685843e+09,2023-06-03 21:48:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,AI regulations in Europe,Intel heads for worst day on Wall Street in 50...,Intel shares plunged the most in 50 years on F...,https://www.reddit.com/r/stocks/comments/1eifw...,430,106,1.722620e+09,2024-08-02 13:26:36,Puginator,stocks,The thing is the average consumer isn't quite ...,NaN,9,1.722621e+09,2024-08-02 13:53:36
920,AI regulations in Europe,Intel heads for worst day on Wall Street in 50...,Intel shares plunged the most in 50 years on F...,https://www.reddit.com/r/stocks/comments/1eifw...,430,106,1.722620e+09,2024-08-02 13:26:36,Puginator,stocks,Please tell me how you’re buying laptops to re...,777IRON,2,1.722624e+09,2024-08-02 14:45:26
921,AI regulations in Europe,Intel heads for worst day on Wall Street in 50...,Intel shares plunged the most in 50 years on F...,https://www.reddit.com/r/stocks/comments/1eifw...,430,106,1.722620e+09,2024-08-02 13:26:36,Puginator,stocks,Had to check to make sure this wasn't Ken M,dinnerthief,1,1.722625e+09,2024-08-02 14:57:42
922,AI regulations in Europe,Intel heads for worst day on Wall Street in 50...,Intel shares plunged the most in 50 years on F...,https://www.reddit.com/r/stocks/comments/1eifw...,430,106,1.722620e+09,2024-08-02 13:26:36,Puginator,stocks,[deleted],NaN,-2,1.722622e+09,2024-08-02 14:03:27


In [49]:
df['post_title']

0       What is so limiting in EU act on AI regulations?
1       What is so limiting in EU act on AI regulations?
2       What is so limiting in EU act on AI regulations?
3       What is so limiting in EU act on AI regulations?
4       What is so limiting in EU act on AI regulations?
                             ...                        
919    Intel heads for worst day on Wall Street in 50...
920    Intel heads for worst day on Wall Street in 50...
921    Intel heads for worst day on Wall Street in 50...
922    Intel heads for worst day on Wall Street in 50...
923    Intel heads for worst day on Wall Street in 50...
Name: post_title, Length: 924, dtype: object

In [50]:
df['post_title'][923]

'Intel heads for worst day on Wall Street in 50 years, falls to lowest in over a decade '